In [51]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AffinityPropagation

from sklearn.metrics import silhouette_score

# Clustering

The task that have to be done corresponds to finding groups of similar products/items, which are dissimilar to each other.

In **Part1**, the similarity matrix was computed. Some of the clustering algorithms require instead the distance matrix. However, this is not a problem, since the distance matrix can be easily obtained as: `dmatrix = 1-smatrix`.

In what follows, 3 clustering algorithms are tested: DBSCAN and Spectral Clustering. Both methods were chosen because for their ability to find clusters of irregular shapes, which probably is the case in our problem (It is not possible visualize it either).


#### Evaluation
In order to evaluate the results from the different clustering algorithms tested, the silhouette metric is used. Silhouette Coefficient is calculated using the mean intra-cluster distance `a` and the mean nearest-cluster distance (b) for each sample. The Silhouette Coefficient for a sample is `(b - a) / max(a, b)`.

The best value is `1` and the worst value is `-1`. Values near `0` indicate overlapping clusters. Negative values generally indicate that a sample has been assigned to the wrong cluster, as a different cluster is more similar.

In [4]:
smatrix = np.load("data/smatrix.npy")
dmatrix = 1-smatrix

### DBSCAN

DBSCAN is a clustering algorithm based on the density of distribution of the data samples. DBSCAN captures the insight that clusters are dense groups of points. The idea is that if a particular point belongs to a cluster, it should be near to lots of other points in that cluster. 

Below DBSCAN is tested with different values of `eps`, which is the radio of neighborhood the algorithm searches for new neighbors. 

In [44]:
eps_list = np.linspace(0., 0.1, 26)[1:]
best_clf = None
best_pred = None
best_score = -1

for eps in eps_list:
    clf = DBSCAN(metric="precomputed", n_jobs=2, eps=eps)
    pred = clf.fit_predict(dmatrix)
    score = silhouette_score(dmatrix, pred, metric="precomputed")
    if score>best_score:
        best_score = score
        best_clf = clf
        best_pred = pred

print("Best silhouette score: ",best_score)
print("Number of clusters:", np.max(best_pred)+1)
print("Predictions:",best_pred)

/Users/martin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)
/Users/martin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)


Best silhouette score:  0.09569720215997334
Number of clusters: 3
Predictions: [ 0  0  0 ... -1 -1 -1]


/Users/martin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)


### Spectral Clustering

Spectral clustering techniques make use of the spectrum (eigenvalues) of the similarity matrix of the data to perform dimensionality reduction before clustering in fewer dimensions.

Spectral Clustering is very useful when the structure of the individual clusters is highly non-convex or more generally when a measure of the center and spread of the cluster is not a suitable description of the complete cluster (**which is clearly our case**).

Below Spectral Clustering is tested with different values of `n_clusters`, which is the radio of neighborhood the algorithm searches for new neighbors. 

In [57]:
n_clusters_list = np.arange(2,21)
best_clf = None
best_pred = None
best_score = -1

for n_cluster in n_clusters_list:
    clf = SpectralClustering(n_clusters=n_cluster, affinity="precomputed", n_jobs=2)
    pred = clf.fit_predict(smatrix)
    score = silhouette_score(dmatrix, pred, metric="precomputed")
    if score>best_score:
        best_score = score
        best_clf = clf
        best_pred = pred

print("Best silhouette score: ",best_score)
print("Number of clusters:", np.max(best_pred)+1)
print("Predictions:",best_pred)

/Users/martin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:709: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  warnings.warn("Array is not symmetric, and will be converted "
/Users/martin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)


Best silhouette score:  0.17174173898115225
Number of clusters: 2
Predictions: [1 1 1 ... 0 0 1]


### Conclusion

Neither of the two methods achieved a great silhouette score, however Spectral Clustering with `n_cluster=2` was the best algorithm configuration. Therefore, this was setting for the `get_cluster.py` script.